# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [98]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [99]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [100]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.prestige, df.admit, margins=True)

admit,0,1,All
prestige,,,
1.0,28,33,61
2.0,95,53,148
3.0,93,28,121
4.0,55,12,67
All,271,126,397


In [101]:
df2 = pd.crosstab(index =df['prestige'],columns=df['admit'])
print df2

admit      0   1
prestige        
1.0       28  33
2.0       95  53
3.0       93  28
4.0       55  12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [102]:
#df['prestige'].astype(str)
# tried to convert to str and then append to get the same naming convention as code below

In [103]:
#dummy_ranks = pd.get_dummies(df['prestige'], drop_first=True, prefix='prestige')
#dummy_ranks.head()

In [104]:
df = df.join(pd.get_dummies(df['prestige'], prefix='prestige_'))
df.head()

,admit,gre,gpa,prestige,prestige__1.0,prestige__2.0,prestige__3.0,prestige__4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need only need 3 for prestige and can drop the first. We can also drop the original prestige column of data since it's now captured in the dummies.

In [105]:
df_clean= df.drop('prestige', axis=1)
df_clean.head()

,admit,gre,gpa,prestige__1.0,prestige__2.0,prestige__3.0,prestige__4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [106]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
#dummy_ranks.head()

cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [107]:
#crosstab prestige 1 admission 
p1admit = pd.crosstab(df['admit'], df['prestige__1.0'], rownames=['admit'])
print p1admit


prestige__1.0    0   1
admit                 
0              243  28
1               93  33


In [108]:
# frequency table cutting prestige and whether or not someone was admitted
pres_admit = pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])
print pres_admit

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [109]:
res1 = 33.0/(33.0+28.0)
print res1

0.540983606557


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [110]:
res2 = 93.0/(243.0+93.0)
print res2

0.276785714286


#### 3.3 Calculate the odds ratio

In [111]:
res3 = 54.1/27.69
print res3

1.9537739256


#### 3.4 Write this finding in a sentenance: 

Answer: Prestige 1 has a 1.95x higher chance of getting admitted.

#### 3.5 Print the cross tab for prestige_4

In [112]:
prestige_admit_tab4 = pd.crosstab(df['admit'], df['prestige__4.0'])
prestige_admit_tab4

prestige__4.0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [113]:
prestige_admit4 = 12./(55.+12.) 
non_prestige_4 = 114./(216.+114.)
odds_ratio = prestige_admit4/non_prestige_4
print odds_ratio

0.518460329929


#### 3.7 Write this finding in a sentence

Answer: You are 50% as likely to get accepted into grad school if you attend a tier 4 school compared to if you went to another school.

## Part 4. Analysis

In [114]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [124]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [125]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [126]:
logit = sm.Logit(data['admit'], data[train_cols])
output = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [127]:
print output.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 25 May 2017   Pseudo R-squ.:                 0.08166
Time:                        19:46:04   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [128]:
params = output.params
conf = output.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: Your odds of getting accepted compared to tier 1 are about 50% as good, but approx 50% better than tier 3 and 4

#### 4.6 Interpret the OR of GPA

Answer: Question does not make sense since it's a continous variable. You could bin this variable and then calculate an OR.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [129]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [130]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [131]:
prestiges = pd.Series(['1.0','2.0', '3.0', '4.0'])
prestiges

#df = pd.DataFrame({'Gene':s.index, 'count':s.values})

0    1.0
1    2.0
2    3.0
3    4.0
dtype: object

In [132]:
prestiges = pd.get_dummies(prestiges, prefix='prestige')
prestiges

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [133]:
int = pd.Series(['1'])
int

0    1
dtype: object

#### 5.2 Make predictions on the enumerated dataset

In [134]:
newmod = cartesian((gres, gpas, prestiges['prestige_2.0'], prestiges['prestige_3.0'], prestiges['prestige_4.0'], int))

In [135]:
predict = output.predict(newmod)
predict

array([ 0.16417268,  0.16417268,  0.16417268, ...,  0.73403998,
        0.73403998,  0.36860803])

In [136]:
df = pd.DataFrame(newmod)
df.head()
df['predict'] = predict
df.tail(4)

,0,1,2,3,4,5,predict
6396,800.0,4.0,0.0,0.0,0.0,1.0,0.734040
6397,800.0,4.0,0.0,0.0,0.0,1.0,0.734040
6398,800.0,4.0,0.0,0.0,0.0,1.0,0.734040
6399,800.0,4.0,0.0,0.0,1.0,1.0,0.368608


#### 5.3 Interpret findings for the last 4 observations

Answer: Probability for the first 3 of getting accepted is 73% and the 4th is 37%.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.